# SQL Demos
Notebook by Chris Pyles, data from Lahman Baseball Dataset

In [12]:
import sqlite3
import pandas as pd
from sqlalchemy import create_engine
db = sqlite3.connect("./lahman2016.sqlite")

look at structure of database

In [13]:
query = """
SELECT * FROM sqlite_master WHERE type = 'table'
"""
pd.read_sql(query, db)

,type,name,tbl_name,rootpage,sql
0,table,AllstarFull,AllstarFull,2,"CREATE TABLE AllstarFull (\nplayerID TEXT,\nye..."
1,table,Appearances,Appearances,3,"CREATE TABLE Appearances (\nyearID INTEGER,\nt..."
2,table,AwardsManagers,AwardsManagers,4,"CREATE TABLE AwardsManagers (\nplayerID TEXT,\..."
3,table,AwardsPlayers,AwardsPlayers,5,"CREATE TABLE AwardsPlayers (\nplayerID TEXT,\n..."
4,table,AwardsShareManagers,AwardsShareManagers,6,CREATE TABLE AwardsShareManagers (\nawardID TE...
5,table,AwardsSharePlayers,AwardsSharePlayers,9,CREATE TABLE AwardsSharePlayers (\nawardID TEX...
6,table,Batting,Batting,10,"CREATE TABLE Batting (\nplayerID TEXT,\nyearID..."
7,table,BattingPost,BattingPost,11,"CREATE TABLE BattingPost (\nyearID INTEGER,\nr..."
8,table,CollegePlaying,CollegePlaying,13,"CREATE TABLE CollegePlaying (\nplayerID TEXT,\..."
9,table,Fielding,Fielding,14,"CREATE TABLE Fielding (\nplayerID TEXT,\nyearI..."


look at years of dataset

In [16]:
query = """
SELECT DISTINCT yearid FROM salaries
"""
pd.read_sql(query, db).T    # transpose to make it look nicer

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
yearID,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016


average salary of hall of famers  in each year in the 5 years with the highest average salaries

In [ ]:
query = """
SELECT s.yearid, AVG(salary) AS avg_salary
FROM salaries s
JOIN halloffame h
ON s.playerid = h.playerid
WHERE h.inducted = 'Y'
GROUP BY s.yearid
ORDER BY avg_salary DESC
LIMIT 5
"""
pd.read_sql(query, db)

calculate slugging percentage of players with more than 50 at-bats:

$$\Large
SLG = \frac{H + 2B + 2 \cdot 3B + 3 \cdot 4B}{AB}
$$

(this is a modified formula because `H` is all hits, not just singles)

In [17]:
db.execute("DROP VIEW IF EXISTS slg")

db.execute("""
CREATE VIEW slg(playerid, yearid, slg) AS
SELECT playerid, yearid, (H + '2B' + 2 * '3B' + 3 * HR) / AB
FROM batting
WHERE AB > 50;
""")

query = """
SELECT * FROM slg;
"""
pd.read_sql(query, db).head()

,playerid,yearid,slg
0,aaronha01,1954,0
1,aaronha01,1955,0
2,aaronha01,1956,0
3,aaronha01,1957,0
4,aaronha01,1958,0


In [18]:
db.execute("DROP VIEW IF EXISTS slg")

db.execute("""
CREATE VIEW slg(playerid, yearid, slg) AS
SELECT playerid, yearid, (CAST(H AS FLOAT) + '2B' + 2 * '3B' + 3 * HR) / AB
FROM batting
WHERE AB > 50;
""")

query = """
SELECT * FROM slg;
"""
pd.read_sql(query, db).head()

,playerid,yearid,slg
0,aaronha01,1954,0.380342
1,aaronha01,1955,0.461794
2,aaronha01,1956,0.469622
3,aaronha01,1957,0.549593
4,aaronha01,1958,0.489185


schools whose players have highest salaries (top 10, names only)

In [19]:
query = """
SELECT name_full
FROM schools c
JOIN (
    SELECT schoolid, AVG(salary) AS salary
    FROM collegeplaying c
    JOIN (
        SELECT playerid, AVG(salary) AS salary
        FROM salaries
        GROUP BY playerid
    ) AS s
    ON c.playerid = s.playerid
    GROUP BY schoolid
) AS s
ON c.schoolid = s.schoolid
ORDER BY s.salary DESC
LIMIT 10
"""
pd.read_sql(query, db).head()

,name_full
0,Holmes Community College
1,University of Hartford
2,University of Wisconsin at Stevens Point
3,University of Illinois at Chicago
4,Maple Woods Community College


what is the YoY percent change in average salary?

In [20]:
query = """
SELECT y2.yearid, (y2.salary - y1.salary) / y1.salary 
FROM (
    SELECT yearid, AVG(salary) AS salary
    FROM salaries
    GROUP BY yearid
    HAVING yearid != (
        SELECT MAX(yearid)
        FROM salaries
    )
) AS y1 
JOIN (
    SELECT yearid, AVG(salary) AS salary
    FROM salaries
    GROUP BY yearid
    HAVING yearid != (
        SELECT MIN(yearid)
        FROM salaries
    )
) AS y2
ON y1.yearid = y2.yearid - 1
"""
pd.read_sql(query, db).head()

,yearid,(y2.salary - y1.salary) / y1.salary
0,1986,-0.124192
1,1987,0.042149
2,1988,0.042421
3,1989,0.117289
4,1990,0.011160
